In [38]:
import argparse
import os


#Pull the data in based on a parameter entered at the command line
parser = argparse.ArgumentParser()
parser.add_argument('--task', '-t', required=True, help='Name of task')
parser.add_argument('--run', '-r', required=True, help='Run number')
parser.add_argument('--subject', '-s', required=True, help='Subject ID number')
#args = parser.parse_args()
#task = args.task
#run = args.run
#subject = args.subject

task = 'WMShapes'
run = 1
subject = 101

#d='/mnt/stressdevlab/dep_threat_pipeline/' + str(subject) + '/' + str(task.lower())
f = []

import os
for task in ['WMShapes','WMFaces','ExtinctionRecall','GNG','ThreatReactivity']:
    d = d='/mnt/stressdevlab/dep_threat_pipeline/' + str(subject) + '/' + str(task.lower())
    for file in os.listdir(d):
        if file.startswith(str(task)) and file.endswith('.txt'):
            if 'csf' not in file and 'SN' not in file and 'wm' not in file and 'regressors' not in file and 'outliers' not in file:
                f.append(str(d) + '/' + str(file))

for i in f:
    print i
#print f

/mnt/stressdevlab/dep_threat_pipeline/101/wmshapes/WMShapes2_low.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmshapes/WMShapes2_high.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmshapes/WMShapes1_low.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmshapes/WMShapes1_Simpson.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmshapes/WMShapes2_Simpson.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmshapes/WMShapes1_high.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmfaces/WMFaces2_angry.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmfaces/WMFaces1_calm.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmfaces/WMFaces2_calm.txt
/mnt/stressdevlab/dep_threat_pipeline/101/wmfaces/WMFaces1_angry.txt
/mnt/stressdevlab/dep_threat_pipeline/101/extinctionrecall/ExtinctionRecall2_CSMinus_Image_R-EMPTY.txt
/mnt/stressdevlab/dep_threat_pipeline/101/extinctionrecall/ExtinctionRecall1_CSPlus_Image_R-EMPTY.txt
/mnt/stressdevlab/dep_threat_pipeline/101/extinctionrecall/ExtinctionRecall1_CSPlus_Image_NR.txt


In [32]:
subject = '999'
SubjectDir = '/mnt/stressdevlab/dep_threat_pipeline/' + '999' + '/'

def ParseExtRecall (data, runData):
    
    #Set string names to match column names in behavioral file
    CSImageStr = 'CSImageRecall'
    ThreatRespStr = 'ThreatResp'
    CSImageStrOnsetTime = str(CSImageStr) + 'OnsetTime'
    CSImageStrOnsetToOnsetTime = str(CSImageStr) + 'OnsetToOnsetTime'
    ThreatRespStrOnsetTime = str(ThreatRespStr) + 'OnsetTime'
    ThreatRespStrOnsetToOnsetTime = str(ThreatRespStr) + 'OnsetToOnsetTime'
    CSImageStrRT = str(CSImageStr) + 'RT'
    ThreatRespStrRT = str(ThreatRespStr) + 'RT'

    #The column name needs to be adjusted when run number = 2
    if (runData['RunNum'] == '2'):
        CSImageStr += '2'
        ThreatRespStr += '2'
        CSImageStrOnsetTime += '2'
        CSImageStrOnsetToOnsetTime += '2'
        ThreatRespStrOnsetTime += '2'
        ThreatRespStrOnsetToOnsetTime += '2'
        CSImageStrRT += '2'
        ThreatRespStrRT += '2'

    #There are two condition in ExtinctionRecall (CSPlus_Threat and CSMinus_Threat)
    for TrialType in ['CSPlus','CSMinus']:
        Condition = TrialType + '_Threat'

        #Get TriggerOnsetTimes for each run and condition
        ImageRecallTrials = data[(data['ProcedureBlock'] == runData['RunName']) & (data['StimulusTrial'] == Condition)]
        TriggerOnsetTimes = ImageRecallTrials.TriggerWAITRTTimeBlock

        #Get ImageOnsets by selecting CSImageRecallOnsetTime and subtracting TriggerOnsetTimes for each run and conditon
        ImageOnsets = (ImageRecallTrials[CSImageStrOnsetTime] - TriggerOnsetTimes) / float(1000)

        #Get ImageDurations from same subset at above (but use CSImageRecallOnsetToOnsetTime column instead)
        ImageDurations = ImageRecallTrials[CSImageStrOnsetToOnsetTime] / float(1000)
           
        #Get ResponseOnsetTime from same subset as above (but use ThreatRespOnsetTime)
        ResponseOnsets = (ImageRecallTrials[ThreatRespStrOnsetTime] - TriggerOnsetTimes) / float(1000)

        #Get ResponseDurations from same subset at above (but use ThreatRespOnsetToOnsetTime)
        ResponseDurations = ImageRecallTrials[ThreatRespStrOnsetToOnsetTime] / float(1000)

        #CombinedDurations are equal to (*RESPONSE* Onsets + Durations) - (*IMAGE* Onsets + Durations)
        CombinedDurations = (ResponseOnsets.values + ResponseDurations.values) - (ImageOnsets.values + ImageDurations.values)
  
        #Get response times from same subset as above (but use ThreatRespRT)
        ResponseTimes = ImageRecallTrials[ThreatRespStrRT] / float(1000)

        t = ImageDurations.add(ResponseDurations)
        print t
        #print ResponseDurations
        
        #Print output for all trials
        #ExtRecallOutput('Image','C', runData, TrialType, ImageOnsets, ImageDurations)
        #ExtRecallOutput('Response','C', runData, TrialType, ResponseOnsets, ResponseDurations)
        #ExtRecallOutput('ImageResponse','C', runData, TrialType, ImageOnsets, CombinedDurations)

for runname in ['ExtinctionRecall1']:
    runData = GetRunData(runname)
    data = pd.read_csv(str(SubjectDir) + 'behavior/' + runData['Task'] + '-eprime.csv')
    #noSimData = FindSimpsons(data, runData)
    ParseExtRecall(data, runData)
    #Touch(str(runData['FullTaskDir'] + 'Onsets.txt'))
			

1     3.950
2     3.983
4     3.933
6     3.933
8     3.933
10    3.933
11    3.933
13    3.966
14    3.966
18    3.933
dtype: float64
0     3.917
3     3.933
5     4.000
7     3.933
9     3.933
12    3.933
15    3.933
16    3.966
17    3.967
19    3.933
dtype: float64


In [55]:
import numpy as np
import argparse
import csv

#Pull the data in based on a parameter entered at the command line
#parser = argparse.ArgumentParser()
#parser.add_argument('--input', '-i', required=True, help='Name of tab-delimited text file to import')
#parser.add_argument('--task', '-t', required=True, help='Task (Threat or Memory)')
#parser.add_argument('--run', '-r', required=True, help='Run number')
#parser.add_argument('--output', '-o', required=True, help='Output path')

#filename = parser.parse_args().input
#Task=parser.parse_args().task
#Run=parser.parse_args().run
#Output=parser.parse_args().output

filename='/mnt/stressdevlab/dep_threat_pipeline/999/behavior/ExtinctionRecall-eprime.csv'
Task='Threat'
Run='1'
Output='/mnt/stressdevlab/dep_threat_pipeline/Test'

#Get the data into an array
data=np.genfromtxt(filename, dtype=None, delimiter=',', names=True)

#Deal with those pesky Simpson trials
CurrentProc='ExtinctionRecall'+Task+str(Run)

#MaggieProc=data[([data['MaggieSimpsonOnsetTime']>0])]
#MargeProc=data[([data['MargeSimpsonOnsetTime']>0])]

#if MaggieProc==CurrentProc:
SimpsonLoc = np.where(data['MaggieSimpsonOnsetTime'] > 0)
print SimpsonLoc
SimpsonOnset = (data[SimpsonLoc]['MaggieSimpsonOnsetTime']-data[SimpsonLoc]['TriggerWAITRTTimeBlock'])/float(1000)
SimpsonDuration = data[SimpsonLoc]['MaggieSimpsonOnsetToOnsetTime']/float(1000)
print str(SimpsonOnset[0]) + str(SimpsonDuration[0]) + str(1)

#elif MargeProc==CurrentProc:
    #SimpsonOnset=((data['MargeSimpsonOnsetTime'][data['MargeSimpsonOnsetTime']>0]-data['TriggerWAITRTTimeBlock'][data['MargeSimpsonOnsetTime']>0])/float(1000))[0]
    #SimpsonDuration=((data['MargeSimpsonOnsetToOnsetTime'][data['MargeSimpsonOnsetTime']>0])/float(1000))[0]

    #print str(SimpsonOnset), str(SimpsonDuration), str(1)
    #OutputFile=open((Output + '/ExtinctionRecall_' + Task[:3] + '_' + str(Run) + '_Simpson.txt'), "w")
    #print(OutputFile)
    #writer = csv.writer(OutputFile, delimiter=' ')		
    #writer.writerow([str(SimpsonOnset), str(SimpsonDuration), str(1)])
    

(array([8]),)
88.9090.9991


In [ ]:

for TrialType in ['CSPlus','CSMinus']:
    Condition=TrialType +'_' + Task
    print(Condition)

    #Get onsets for Response & NonResponse Combined
    ImageOnsets=(data['CSImageRecallOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))]-data['TriggerWAITRTTimeBlock'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])/float(1000)
    ImageDurations=(data['CSImageRecallOnsetToOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])/float(1000)

    ResponseOnsets=(data[Task+'RespOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))]-data['TriggerWAITRTTimeBlock'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])/float(1000)
    ResponseDurations=(data[Task+'RespOnsetToOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])/float(1000)
    
    CombinedDurations=(data[Task+'RespOnsetToOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))] + data[Task+'RespOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))] - data['CSImageRecallOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])/float(1000)

    ResponseTimes=(data[Task+'RespRTTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])
    
    OutputFile=open(Output + '/ExtinctionRecall' + str(Run) + '_' + TrialType + '_Image.txt', "w")
    writer = csv.writer(OutputFile, delimiter=' ')
    for row in range(0,len(ImageOnsets)):
        writer.writerow([str(ImageOnsets[row]), str(ImageDurations[row]), str(1)])
        
    OutputFile=open(Output + '/ExtinctionRecall' + str(Run) + '_' + TrialType + '_Response.txt', "w")
    writer = csv.writer(OutputFile, delimiter=' ')
    for row in range(0,len(ResponseOnsets)):
        writer.writerow([str(ResponseOnsets[row]), str(ResponseDurations[row]), str(1)])
        
    OutputFile=open(Output + '/ExtinctionRecall' + str(Run) + '_' + TrialType + '_ImageResponse.txt', "w")
    writer = csv.writer(OutputFile, delimiter=' ')
    for row in range(0,len(ImageOnsets)):
        writer.writerow([str(ImageOnsets[row]), str(CombinedDurations[row]), str(1)])
        print(row)
        
    #Get Onsets by Response Type
    NRImageOnsets=ImageOnsets[ResponseTimes==0]
    NRImageDurations=ImageDurations[ResponseTimes==0]
    NRResponseOnsets=ResponseOnsets[ResponseTimes==0]
    NRResponseDurations=ResponseDurations[ResponseTimes==0]
    NRCombinedDurations=CombinedDurations[ResponseTimes==0]
    
    if len(NRImageOnsets)>0:
        OutputFile=open(Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_Image_NR.txt', "w")
        writer = csv.writer(OutputFile, delimiter=' ')
        for row in range(0,len(NRImageOnsets)):
            writer.writerow([str(NRImageOnsets[row]), str(NRImageDurations[row]), str(1)])
            
        OutputFile=open(Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_Response_NR.txt', "w")
        writer = csv.writer(OutputFile, delimiter=' ')
        for row in range(0,len(NRResponseOnsets)):
            writer.writerow([str(NRResponseOnsets[row]), str(NRResponseDurations[row]), str(1)])

        OutputFile=open(Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_ImageResponse_NR.txt', "w")
        writer = csv.writer(OutputFile, delimiter=' ')
        for row in range(0,len(NRImageOnsets)):
            writer.writerow([str(NRImageOnsets[row]), str(NRCombinedDurations[row]), str(1)])
            
    RImageOnsets=ImageOnsets[ResponseTimes>0]
    RImageDurations=ImageDurations[ResponseTimes>0]
    RResponseOnsets=ResponseOnsets[ResponseTimes>0]
    RResponseDurations=ResponseDurations[ResponseTimes>0]
    RCombinedDurations=CombinedDurations[ResponseTimes>0]

    if len(RImageOnsets)>0:
        OutputFile=open(Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_Image_R.txt', "w")
        writer = csv.writer(OutputFile, delimiter=' ')
        for row in range(0,len(RImageOnsets)):
            writer.writerow([str(RImageOnsets[row]), str(RImageDurations[row]), str(1)])

        OutputFile=open(Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_Response_R.txt', "w")	
        writer = csv.writer(OutputFile, delimiter=' ')
        for row in range(0,len(RResponseOnsets)):
            writer.writerow([str(RResponseOnsets[row]), str(RResponseDurations[row]), str(1)])

        OutputFile=open(Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_ImageResponse_R.txt', "w")
        writer = csv.writer(OutputFile, delimiter=' ')
        for row in range(0,len(RImageOnsets)):
            writer.writerow([str(RImageOnsets[row]), str(RCombinedDurations[row]), str(1)])
                
                
    #Calculate Response percentage 
    ResponseRate=np.array(float(len(data['CSImageRecallOnsetTime'][np.logical_and(np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run))),data[Task+'RespRT']>0)]))/len(data['CSImageRecallOnsetTime'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])*float(100))
    print(ResponseRate)
    np.savetxt(Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_PercentResponse.txt',ResponseRate.reshape(1),fmt='%s')

    #Calculate Rating
    MeanRating=np.mean(data['ThreatRating'][np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run)))])
    print(MeanRating)
    np.savetxt((Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_MeanRating.txt'),MeanRating.reshape(1),fmt='%s')

    #Calculate RT
    RT=np.mean(data['ThreatRespRT'][np.logical_and(np.logical_and(data['StimulusTrial']==Condition,data['ProcedureBlock']==('ExtinctionRecallRun'+str(Run))),data['ThreatRespRT']>0)])
    print(RT)
    np.savetxt((Output+'/ExtinctionRecall' + str(Run) + '_' + TrialType + '_MeanRT.txt'),RT.reshape(1),fmt='%s')
